In [1]:
import pandas as pd
pd.set_option('display.max_columns', None) # set the max columns to display to none
#Private data source: https://www.sbe.wa.gov/our-work/private-schools
d2018 = pd.read_excel('WA_privateschools/Final Excel P105B Combined Data for 201819.xlsx')
d2019 = pd.read_excel('WA_privateschools/Copy of 2019-20 Private School Enrollment_073120.xlsx', sheet_name=1, skiprows=[0,1])
d2020 = pd.read_excel('WA_privateschools/Copy of 2020-21 Private School Enrollment x Grade_for the website.xlsx')
d2021 = pd.read_excel('WA_privateschools/2021-22 Private School Enrollment (Website).xlsx')

regionSubstring = "ellevue"  # Bellevue et al.  Replace with other districts and re-run if you wish

In [2]:
gradeMap = {'Total PreK':'P',
 'Total KG':'K',
 'Total G1':'1',
 'Total G2':'2',
 'Total G3':'3',
 'Total G4':'4',
 'Total G5':'5',
 'Total G6':'6',
 'Total G7':'7',
 'Total G8':'8',
 'Total G9':'9',
 'Total G10':'10',
 'Total G11':'11',
 'Total G12':'12',
 'PreK': 'P',
 'KG': 'K',
 'Grade 1':'1',
 'Grade2':'2',
 'Grade 2':'2',
 'Grade 3':'3',
 'Grade 4':'4',
 'Grade 5':'5',
 'Grade 6':'6',
 'Grade 7':'7',
 'Grade 8':'8',
 'Grade 9':'9',
 'Grade 10':'10',
 'Grade 11':'11',
 'Grade 12':'12',            
  1:'1', 2:'2', 3:'3', 4:'4', 5:'5', 6:'6', 7:'7', 8:'8', 9:'9', 10:'10', 11:'11', 12:'12'}

# Final columns, except Grade, Total
datacols = [ 
  'School Name',
  # 'Street Address', # Missing in some
  # Some are "District Name", some are "City"
  # Will use to hold both and search for substrings: "Bellevue" 
  "Region",
  # 'State',
  # 'Zipcode', Missing in 2018
  # 'County'
  'Year', # Added 
]
allFinalCols = datacols + ["Grade", "Total"]
# Final grade categories
grades= ["P", "K"] + [str(g) for g in range(1, 13)]

In [3]:
# Normalize 2018 --- Not used. Format is quite a lot difference. Suspicious of the changes
# d2018_ = d2018.rename(columns={"District Name": "Region"}) # "City" is missing in this dataset. We only have "District Name"
# d2018_['Grade'] = d2018['Grade'].replace(gradeMap)
# d2018_['Year'] = 2018
# d2018_ = d2018_[d2018_['School Name'].notnull()]
# print(d2018_.shape, list(d2018_[allFinalCols].columns))

In [4]:
# Normalize 2019
df = d2019.rename(columns=gradeMap)
df = df.rename(columns={"Total":"_Total", "Name of School": "School Name", "City": "Region"})
# print(sorted(df.columns))
df = df.melt(id_vars=set(df.columns)-set(grades), value_vars=grades, var_name="Grade", value_name="Total")
df['Year'] = 2019
df = df[df['School Name'].notnull()]
print(df.shape, list(df[allFinalCols].columns))
d2019_ = df

(6790, 480) ['School Name', 'Region', 'Year', 'Grade', 'Total']


In [5]:
# Normalize 2020
df = d2020.rename(columns=gradeMap).rename(columns={"City": "Region"})
df[grades]  # The grade columns should all exist
df = df.melt(id_vars=set(df.columns)-set(grades), value_vars=grades, var_name="Grade", value_name="Total")
df['Grade'] = df['Grade'].replace(gradeMap)
df['Year'] = 2020
df = df[df['School Name'].notnull()]
print(df.shape, list(df[allFinalCols].columns))
d2020_ = df

(7182, 10) ['School Name', 'Region', 'Year', 'Grade', 'Total']


In [6]:
# Normalize 2021
df = d2021.rename(columns=gradeMap).rename(columns={"Name of School": "School Name", 'ZIP': "Zipcode", "City": "Region"})
df[grades]  # The grade columns should all exist
df = df.melt(id_vars=set(df.columns)-set(grades), value_vars=grades, var_name="Grade", value_name="Total")
df['Grade'] = df['Grade'].replace(gradeMap)
df['Year'] = 2021
df = df[df['School Name'].notnull()]
print(df.shape, list(df[allFinalCols].columns))
d2021_ = df

(6944, 22) ['School Name', 'Region', 'Year', 'Grade', 'Total']


In [7]:
# Normalize 2022
# df = d2022.rename(columns=gradeMap).rename(columns={"Name of School": "School Name", 'ZIP': "Zipcode", "City": "Region"})
# df[grades]  # The grade columns should all exist
# df = df.melt(id_vars=set(df.columns)-set(grades), value_vars=grades, var_name="Grade", value_name="Total")
# df['Grade'] = df['Grade'].replace(gradeMap)
# df['Year'] = 2022
# df = df[df['School Name'].notnull()]
# print(df.shape, list(df[allFinalCols].columns))
# d2022_ = df

In [8]:
# 2018 seems to have some very different reporting. Not sure it is consistent
dAll = pd.concat([d2019_, d2020_, d2021_]).reindex()
# Place the important data in the first columns. 
dAll = dAll[allFinalCols + list(set(dAll.columns)-set(allFinalCols))].sort_values(["Year", "Grade", "School Name"])
dAll = dAll[~(dAll["School Name"].isnull())] # Zap rows without school names
dAll = dAll[~(dAll["School Name"].astype(str)=='NaN')] # Zap rows without school names
dAll = dAll[~(dAll["School Name"].isna())] # Zap rows without school names
dAll = dAll[dAll["School Name"]!="State Total"] # Zap total rows
# There are a number of school in the data without a district or city, or a district of 0 for some reason.
# We'll just ignore them from bellevue processing since none are in Bellevie
# dAll = dAll[dAll["Region"].astype(str)!='0'] # 0 is not a region
print("Schools reporting their city/district as 0")
display(dAll[dAll["Region"].isnull()]["School Name"].unique())

print("\nTotal WA private enrollment by year")
display(dAll.groupby("Year")["Total"].sum())

Schools reporting their city/district as 0


array(['Carden Country School', 'Cascades Montessori Middle School',
       'Cedar Park Christian School',
       "Children's Garden Montessori School", 'Gateway Elementary School',
       'Gersh Academy Cougar Mountain', 'Holy Family Parish School',
       'MMSC Day School', 'Northwest Montessori School',
       "O'Dea High School", 'Rainier Christian',
       'Rainier Christian High School', 'Rainier Christian Middle School',
       'Rainier Christian-Maple Valley Elementary',
       'Sonshine Christian Elementary School',
       'Spokane Valley Adventist School', 'St. Pius X School',
       'St. Therese Catholic Academy',
       'The Downtown School a Lakeside School', 'The Franklin Academy',
       'Valley Christian School'], dtype=object)


Total WA private enrollment by year


Year
2019    73608.0
2020    74856.0
2021    88085.0
Name: Total, dtype: float64

In [9]:
print(f'Regions considered:')

dAllRegionOfInterest = dAll[~(dAll["Region"].isnull())]
dAllRegionOfInterest = dAllRegionOfInterest[dAllRegionOfInterest["Region"].astype(str).str.contains(regionSubstring)]
display(dAllRegionOfInterest["Region"].unique())

print(f'\nSchools reporting: {dAllRegionOfInterest["School Name"].unique().size}.')
display(list(dAllRegionOfInterest["School Name"].unique()))
zeroReportedAnyYear = dAllRegionOfInterest.groupby("School Name")["Total"].sum()
zeroReportedAnyYear = zeroReportedAnyYear[zeroReportedAnyYear==0]

print(f'\nSchool reporting 0 enrollment (any year): {zeroReportedAnyYear.size}')
display(zeroReportedAnyYear)

Regions considered:


array(['Bellevue'], dtype=object)


Schools reporting: 40.


["America's Child Montessori",
 "Bellevue Children's Academy",
 'Bellevue Montessori School',
 'Cedar Park Christian School',
 'Chestnut Hill Academy',
 'Dartmoor School',
 'Eastside Academics School',
 'Eastside Academy',
 'Eastside Christian School',
 'Eastside Community School',
 'Emerald Heights Academy',
 'Eton School',
 'French Immersion School of Washington',
 'Hillside Student Community School',
 'International Friends School ',
 'International Montessori Academy',
 'Living Montessori',
 'Medina Academy',
 'Open Window School',
 "SARODGINI CHILDREN'S ACADEMY",
 'Sacred Heart School',
 'Seattle VocTech School',
 'St. Louise School',
 'St. Madeleine Sophie School',
 'The ArtSci Company',
 'The Jewish Day School Of Metropolitan Seattle',
 'The Little School',
 'Bel-Red Bilingual Academy',
 'Forest Ridge School of the Sacred Heart',
 'Asia Pacific Language School',
 'Cedar Crest Academy, Bellewood',
 'Cedar Park Christian - Bellevue',
 'Dartmoor School - Bellevue',
 'International 


School reporting 0 enrollment (any year): 3


School Name
International Friends School     0.0
The ArtSci Company               0.0
Ventures Academy                 0.0
Name: Total, dtype: float64

In [11]:
# Show enrollment changes from baseline year
baselineYear = 2019
for name, select in [
    ("P-3", ["P", "K", "1", "2", "3"]),
    ("K-5", ["K", "1", "2", "3"," 4", "5"]),
    ("P-5", ["P", "K", "1", "2", "3", "4", "5"]),
    ("K-3", ["K", "1", "2", "3"]),
    ("K-12", ["K"]+[str(s) for s in range(1,13)]),
    ("P-12", ["P","K"]+[str(s) for s in range(1,13)]),
  ] + [(g, [g]) for g in grades]:
    
  bvDiff = dAllRegionOfInterest[dAllRegionOfInterest["Grade"].isin(select)]
  bvDiffTotals = bvDiff.groupby("Year")["Total"].sum().to_frame()
  totalName = f"'Grade {name}' private enrollment, Δ since {baselineYear}"
  bvDiffTotals[totalName] = bvDiffTotals["Total"] - bvDiffTotals["Total"][baselineYear]
  bvDiffTotals[f"%change"] = 100*(bvDiffTotals[totalName] / bvDiffTotals["Total"][baselineYear])
  
  g = bvDiff.groupby(["School Name", "Year"])["Total"].sum().to_frame().reset_index()
      
  bvDiffTotals[f"# schools reporting"] = g.groupby(["Year"]).count()["Total"]
  bvDiffTotals[f"# reporting 0"] = g[g["Total"]==0].groupby(["Year"]).count()["Total"]
  bvDiffTotals[f"# reporting not 0"] = g[g["Total"]!=0].groupby(["Year"]).count()["Total"]

  display(bvDiffTotals)

print("Full data")
with pd.option_context('display.max_rows', None, 'display.max_columns', None):    
  display(dAllRegionOfInterest[["School Name", "Grade", "Year", "Total"]].sort_values(["School Name", "Grade", "Year"]))

,Total,"'Grade P-3' private enrollment, Δ since 2019",%change,# schools reporting,# reporting 0,# reporting not 0
Year,,,,,,
2019,1652.0,0.0,0.000000,27,10,17
2020,2302.0,650.0,39.346247,28,9,19
2021,2934.0,1282.0,77.602906,31,6,25


,Total,"'Grade K-5' private enrollment, Δ since 2019",%change,# schools reporting,# reporting 0,# reporting not 0
Year,,,,,,
2019,1608.0,0.0,0.000000,27,9,18
2020,2304.0,696.0,43.283582,28,6,22
2021,2555.0,947.0,58.893035,31,4,27


,Total,"'Grade P-5' private enrollment, Δ since 2019",%change,# schools reporting,# reporting 0,# reporting not 0
Year,,,,,,
2019,2280.0,0.0,0.000000,27,9,18
2020,3139.0,859.0,37.675439,28,6,22
2021,3718.0,1438.0,63.070175,31,4,27


,Total,"'Grade K-3' private enrollment, Δ since 2019",%change,# schools reporting,# reporting 0,# reporting not 0
Year,,,,,,
2019,1309.0,0.0,0.000000,27,10,17
2020,1860.0,551.0,42.093201,28,9,19
2021,2188.0,879.0,67.150497,31,6,25


,Total,"'Grade K-12' private enrollment, Δ since 2019",%change,# schools reporting,# reporting 0,# reporting not 0
Year,,,,,,
2019,2606.0,0.0,0.000000,27,7,20
2020,4648.0,2042.0,78.357636,28,3,25
2021,4189.0,1583.0,60.744436,31,1,30


...

,Total,"'Grade 10' private enrollment, Δ since 2019",%change,# schools reporting,# reporting 0,# reporting not 0
Year,,,,,,
2019,19.0,0.0,0.000000,27,23,4
2020,178.0,159.0,836.842105,28,22,6
2021,93.0,74.0,389.473684,31,24,7


,Total,"'Grade 11' private enrollment, Δ since 2019",%change,# schools reporting,# reporting 0,# reporting not 0
Year,,,,,,
2019,27.0,0.0,0.000000,27,23,4
2020,144.0,117.0,433.333333,28,22,6
2021,72.0,45.0,166.666667,31,24,7


,Total,"'Grade 12' private enrollment, Δ since 2019",%change,# schools reporting,# reporting 0,# reporting not 0
Year,,,,,,
2019,34.0,0.0,0.000000,27,23,4
2020,361.0,327.0,961.764706,28,22,6
2021,65.0,31.0,91.176471,31,25,6


Full data


,School Name,Grade,Year,Total
1042,America's Child Montessori,1,2019,3.0
1032,America's Child Montessori,1,2020,7.0
999,America's Child Montessori,1,2021,10.0
5416,America's Child Montessori,10,2019,0.0
5649,America's Child Montessori,10,2020,0.0
5463,America's Child Montessori,10,2021,0.0
5902,America's Child Montessori,11,2019,0.0
6162,America's Child Montessori,11,2020,0.0
5959,America's Child Montessori,11,2021,0.0
6388,America's Child Montessori,12,2019,0.0
